# Parse xyz file

## Initialize environment

In [1]:
import numpy as np

## Read and store the molecule information

In [2]:
file_106080 = open("dsgdb9nsd_106080.xyz",'r').read().split('\n')
data_106080 = [ i.split() for i in file_106080 ]

Here is all the data in the molecule where index is 106080:

In [3]:
data_106080

[['19'],
 ['gdb',
  '106080',
  '2.62335',
  '1.58418',
  '1.36308',
  '1.6257',
  '82.15',
  '-0.2132',
  '-0.0367',
  '0.1765',
  '1034.7611',
  '0.159626',
  '-385.782619',
  '-385.774784',
  '-385.77384',
  '-385.814995',
  '30.93'],
 ['O', '0.3767764618', '1.2859110586', '-0.4697578875', '-0.41404'],
 ['C', '0.1549985521', '-0.0830289164', '-0.1495922755', '-0.203274'],
 ['C', '-1.3001970311', '-0.4129308688', '-0.0418817809', '0.305177'],
 ['C', '-2.0488745331', '-1.7141397759', '0.0352968618', '-0.136793'],
 ['C', '-2.4940655422', '-1.4389304636', '1.5203476238', '-0.264865'],
 ['C', '-2.2734333004', '0.0176797313', '1.0818951199', '-0.019136'],
 ['C', '-3.3467300513', '0.660777344', '0.5781146096', '-0.098105'],
 ['C', '-3.3671101442', '0.5109088159', '-0.940875516', '-0.264323'],
 ['C', '-2.3358269261', '-0.6236883079', '-1.0836225241', '-0.088446'],
 ['H', '-0.0580943522', '1.8094228593', '0.2119971258', '0.284797'],
 ['H', '0.6113845427', '-0.6601113335', '-0.9612080656', '0

Then we need to convert floats and integers. We first define a function converting string to integer and float64.

In [4]:
def str2intflt(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return s

Then we convert the values in `data_106080` to integers and floats

In [5]:
for Indi, i in enumerate(data_106080):
    for Indj, j in enumerate(i):
        data_106080[Indi][Indj] = str2intflt(j)

# Feature Extraction

## CM

CM (Coulumb Matrix) method is the easiest to be parsed.

Though the original idea of CM was proposed by Rupp *et al.* ([PRL 2012](https://dx.doi.org/10.1103/PhysRevLett.108.058301)); however, for the realization in this article Faber *et al.* ([JCTC 2017](https://dx.doi.org/10.1021/acs.jctc.7b00577)), the definition of similarity space has been changed.

Firstly, the feature vector is $30 \times 30$ length. Every element refers to a specific atom (if there be filled an atom, or 0 otherwise).

Secondly, the off-diagonal elements of the matrix is evaluated by the classic electrostatic force by neucleu:
$$
M_{AB} = \frac{Z_A Z_B}{| R_A - R_B |}
$$
The diagonal elements are evaluated by the following fomular:
$$
M_{AA} = 0.5 Z_A^{2.4}
$$

Finally, the columns and rows (refer to the atoms in principle) are sorted by their $L_1$ norms in decreasing order.

We now try to parse xyz data to get the CM feature vector.

(1) Read the coordinate matrix, as well as convert atom name to its neucleu charge.

In [6]:
Coord = np.array([ i[1:4] for i in data_106080[2:-4] ])

In [7]:
# Since replacement of atoms can be done manually (C, N, O, F, H),
# so I just do replacement uncleverly.
NeuZ = np.array( [ int(i[0]
    .replace('C','6').replace('N','7').replace('O','8') \
    .replace('F','9').replace('H','1')) \
    for i in data_106080[2:-4] ] )

(2) Build up the elements of CM.

In [8]:
AtomNum = NeuZ.shape[0]
CM = np.zeros((30, 30))
for i in range(AtomNum):
    CM[i][i] = 0.5 * np.power(NeuZ[i], 2.4)
    for j in range(i+1, AtomNum):
        temp_float = NeuZ[i] * NeuZ[j] / np.linalg.norm(Coord[i] - Coord[j])
        CM[i][j] = temp_float
        CM[j][i] = temp_float

(3) Sort columns and rows by the $L_2$ norm of CM vectors.

> Note that in [Faber *et al.* (2017)](https://dx.doi.org/10.1021/acs.jctc.7b00577), they say that the matrix is sorted by $L_1$ norm. This is probably wrong at least in this molecule.

In [9]:
#CM_Norm = np.array([ abs(i).sum() for i in CM ])
CM_Norm = np.array([ np.linalg.norm(i) for i in CM ])

In [10]:
# Following code is learned form
# https://stackoverflow.com/questions/25539593/sort-a-list-then-give-the-indexes-of-the-elements-in-their-original-order
CM_Sort = [ b[0] for b in sorted(enumerate(CM_Norm), key=lambda i:-i[1]) ]

In [11]:
CM = CM[CM_Sort]
CM = np.transpose(CM)[CM_Sort]

(4) Finally, we concate CM to a one-dimension array.

In [12]:
CM = np.concatenate(CM)

(5) Compare the CM matrix we created to that from Faber *et al.* (2017)

In [13]:
CM_Confirm = np.array([ float(i) for i in \
    open("./qm9_106080/CM_106080.txt",'r').read().split()[1:] ])

In [14]:
np.linalg.norm(CM - CM_Confirm)

2.0316240359242251e-10

This deviation is so small that we can ignore that.

## BoB

### Location of bags

We first need to know the intial location of bags. These locations can be obtained by observation on some special molecules.

|Molecules | QM9 Index | Atoms & Bonds | Location
|:--------:|:---------:|:-------------:|---------:
|    CH4 | 000001 | C   | 0
|        |        | H   | 9
|        |        | C-H | 319
|        |        | H-H | 83
|    NH3 | 000002 | N   | 29
|        |        | N-H | 661
|    H2O | 000003 | O   | 36
|        |        | O-H | 801
|   C2H2 | 000004 | C-C | 47
|    HCN | 000005 | C-N | 499
|   CH2O | 000006 | C-O | 562
| NH2CHO | 000012 | O-N | 1021
|CO(NH2)2| 000020 | N-N | 273
| CHOCHO | 000028 | O-O | 294
|    CF4 | 000184 |   F | 41
|        |        | C-F | 607
|        |        | F-F | 304
| CF3CCH | 000826 | H-F | 901
|  CF3CN | 000827 | N-F | 1056
| CF3CHO | 000828 | O-F | 1098

The following code is utilized to find these locations.

In [15]:
Temp_Str = open("./BOB_example/BOB_000828",'r').read().split('\n')[0].split()[1:]
print("Length of this input array: %s " % len(Temp_Str))
print("The following array is the non-zero value index in array:")
Temp_List = [ Indi for Indi, i in enumerate(Temp_Str) if i != "0.0" ]
print(Temp_List)

print("Length of non-zero elements: %s" % len(Temp_List))

Length of this input array: 2209 
The following array is the non-zero value index in array:
[0, 1, 9, 36, 41, 42, 43, 47, 304, 305, 306, 319, 320, 562, 563, 607, 608, 609, 610, 611, 612, 801, 901, 902, 903, 1098, 1099, 1100]
Length of non-zero elements: 28


Or we can write the location as following:

 Atoms & Bonds | Location
:-------------:|---------:
C   | 0
H   | 9
N   | 29
O   | 36
F   | 41
C-C | 47
H-H | 83
N-N | 273
O-O | 294
F-F | 304
C-H | 319
C-N | 499
C-O | 562
C-F | 607
N-H | 661
O-H | 801
F-H | 901
N-O | 1021
N-F | 1056
O-F | 1098

It is probably that the 2209 length array is too long for BoB feature vectors. The largest index of non-zero values in these arrays may probably not exceded to 1200.

It's also notable that though BoB vectors are times larger than CM vectors, however, the non-zero value in BoB vectors are $ N \times (N+1) / 2 $, while CM vectors are $ N^2 $; so the compressed zip file of BoB is smaller than CM.

### BoB feature vector

Actually, elements of BoB feature vectors are equalivent to that of CM feature vectors. The only difference is all the elements are packed in "bags" by the kind of bonds or atoms. Inside of these bags, the values are sorted descendingly.

(1) Like CM, read the coordinate matrix, as well as convert atom name to its neucleu charge.

In [16]:
Coord = np.array([ i[1:4] for i in data_106080[2:-4] ])

In [17]:
# Since replacement of atoms can be done manually (C, N, O, F, H),
# so I just do replacement uncleverly.
NeuZ = np.array( [ int(i[0]
    .replace('C','6').replace('N','7').replace('O','8') \
    .replace('F','9').replace('H','1')) \
    for i in data_106080[2:-4] ] )

(2) Create lists of bags.

In [18]:
# - NeuAll: All the neucleu charges in a list
NeuAll = [6, 1, 7, 8, 9]
# - NeuInd: Index of the one or two atom pairs
#           Up to now, the index can be defined by (i+10)*(j+10).
#           However, more atoms or more triples, quards may invalid.
NeuInd = [16, 11, 17, 18, 19, 256, 121, 289, 324, 361,\
    176, 272, 288, 304, 187, 198, 209, 306, 323, 342]

There are totally 20 kind of bonds (atoms). We need to create a 20\*None empty arrows.

In [19]:
Temp_BoB = [ [] for i in range(20) ]

Initial locations are defined by the following list.

In [20]:
NeuLoc = [0, 9, 29, 36, 41, 47, 83, 273, 294, 304, 319, 499, 562, 607, 661, 801, 901, 1021, 1056, 1098]

(3) Fill in values to `Temp_BoB`.

Note that the distance is defined by atomic unit (a.u.). So the unit of length of bonds in QM9 file (angstrom, $ 10^{-10} $ m) should be converted to a.u.. Since 1 a.u. of length is Bohr radius ($ \simeq 0.529177 \times 10^{-10} $ m), so we just simply divide the length of bond by 0.52917724888.

So the values of BoB feature vectors are not equal to that of CM. All the atoms are the same, but all the bonds are multipled by 0.52917724888.

In [21]:
for (Indi, i) in enumerate(Coord):
    Val_CM = 0.5*np.power(NeuZ[Indi], 2.4)
    Temp_BoB[NeuInd.index(NeuZ[Indi]+10)].append(Val_CM)
    for (Indj_subi, j) in enumerate(Coord[Indi+1:]):
        Indj = Indj_subi + Indi + 1
        Val_CM = NeuZ[Indi] * NeuZ[Indj] / (np.linalg.norm(i - j) / 0.52917724888)
        Temp_BoB[NeuInd.index((NeuZ[Indi]+10)*(NeuZ[Indj]+10))].append(Val_CM)

Then we need to reindex the lists in `Temp_BoB`.

In [22]:
for (Indi, i) in enumerate(Temp_BoB):
    i.sort(reverse=True)

(4) Fill in values to the actual BoB feature vector.

In [23]:
BoB = np.zeros(2209)
for (Indi, i) in enumerate(Temp_BoB):
    BoB[NeuLoc[Indi]:NeuLoc[Indi]+len(i)] = i

(5) Compare the BoB vector we created to that from Faber et al. (2017)

In [24]:
BoB_Confirm = np.array([ float(i) for i in \
    open("./qm9_106080/BOB_106080.txt",'r').read().split()[1:] ])

In [25]:
np.linalg.norm(BoB - BoB_Confirm)

2.5693369947722708e-10